## **Estimation of freshwater lens thickness (_FWL_) using three different methods**

> **Objective:** Apply three different methods to estimate the depth of the freshwater lens. Based on this estimation, extract the corresponding array for the freshwater zone, calculate basic statistics, and plot the boxplots for each profile using the three methods.


---

### Import Libraries

In [5]:
import sys
import os

root = os.path.abspath('..')  
sys.path.append(root)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import modules.sec_fwl_estimation as sec_fwl

from modules import processing, load, plots, analysis

---

### Load data

1. Cargar datos del primer método

---

### **Method 1:** _Intuitive criterion_